This notebook describes the temporal fusion transformers [@lim2021temporal] architecture, and ports it over to keras 3 while making some punctual improvements.

The original repository is: https://github.com/google-research/google-research/tree/master/tft.

In [1]:
from __future__ import annotations

import numpy as np
import keras_core as keras
from keras_core import layers
from fastcore import docments
from nbdev.showdoc import show_doc


Using TensorFlow backend


2023-10-11 21:35:34.703677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Architecture

## Concepts

* **time distributed**: 
  * applies same layer to each of the timesteps in the data
    * in other words, a layer with the exact same weights
  * indices:
    * index 0: batch
    * index 1: time
    * indices 2...: data
  * More info: https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed

# Gated residual network

## Linear layer

* dedicated implementation to better control use of time distribution

In [2]:
def linear_layer(size:int, # Output size
                 activation:str|callable|None=None, # Activation function
                 use_time_distributed:bool=False, # Apply the layer across all timesteps?
                 use_bias:bool=True # Include bias in the layer?
)->keras.src.layers.core.dense.Dense: # Dense layer
    "Linear layer."

    linear = keras.layers.Dense(size, activation=activation, use_bias=use_bias)
    if use_time_distributed:
        linear = keras.layers.TimeDistributed(linear)
    return linear

## Dense layer

* dedicated implementation to better control use of time distribution

In [3]:
def dense_layer(
    size:int, # Output size
    activation:str|callable|None=None, # Activation function
    use_time_distributed:bool=False, # Apply the layer across all timesteps?
    use_bias:bool=True # Include bias in the layer?
)->keras.src.layers.core.dense.Dense: # Dense layer
    "Dense layer"

    dense = layers.Dense(size, activation=activation, use_bias=use_bias)
    if use_time_distributed:
        dense = layers.TimeDistributed(dense)
    return dense

Example usage of dense layer:

In [4]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 100
layer_size = 16

# input dimensions: batches / timesteps / features
x = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# dense layer
dense = dense_layer(size=layer_size, use_time_distributed=True)

# output dimensions: batches / timesteps / layer size
assert dense(x).shape == [batch_size, n_timesteps, layer_size]

2023-10-11 21:35:38.539842: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-11 21:35:38.539886: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Now showing that the time-distributed layer applies the same weights at all timesteps:

In [5]:
#| code-fold: show

x = np.ones((1, n_timesteps, n_features))
timesteps_equal = []
for i in range(n_timesteps-1):
    timesteps_equal.append((np.array_equal(dense(x)[0,0,:], dense(x)[0,i+1,:])))

assert np.all(timesteps_equal)

In [6]:
#| output: asis
show_doc(dense_layer)


---

### dense_layer



Dense layer

## Gated linear unit (GLU)

* Introduced by @dauphin2017language
* The GLU is part of the Gated Residual Network (GRN) block
* Using input $\gamma \in \mathbb{R}^{d_{\text{model}}}$ and the subscript $\omega$ to index weights, $\text{GLU}_{\omega}(\gamma) = \sigma(W_{4, \omega} \gamma + b_{4, \omega}) \odot (W_{5, \omega} \gamma + b_{5, \omega})$
* As can be seen above, the result could be very close to zero through the Hadamard multipliciation, which in practice means that the network would not be affected by that data (ie, it would be gated out)
* *"GLUs reduce the vanishing gradient problem for deep architectures by providing a linear path for gradients while retaining non-linear capabilities"*
* *"provide flexibility to suppress any parts of the architecture that are not required for a given dataset"*


In [7]:
#| output: false

def apply_gating_layer(
    x, # Input tensors (batch first)
    hidden_layer_size:int, # Dimension of the GLU
    dropout_rate:float|None=None, # Dropout rate
    use_time_distributed:bool=True, # Apply the GLU across all timesteps?
    activation:str|callable=None # Activation function
): # Tuple of (GLU output tensors, gated_layer)
    "Gated Linear Unit (GLU) layer"
    
    if dropout_rate is not None:
        x = layers.Dropout(dropout_rate)(x)

    activation_layer = layers.Dense(
                hidden_layer_size,
                activation=activation
            )

    gated_layer = layers.Dense(
                hidden_layer_size,
                activation='sigmoid'
            )

    if use_time_distributed:
        activation_layer = layers.TimeDistributed(activation_layer)(x)
        gated_layer = layers.TimeDistributed(gated_layer)(x)
    else:
        activation_layer = activation_layer(x)
        gated_layer = gated_layer(x)

    return layers.Multiply()([activation_layer, gated_layer]), gated_layer

Example usage of GLU:

In [8]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 100
hidden_layer_size = 16

# input dimensions: batches / timesteps / features
x = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# output dimensions: batches / timesteps / hidden_layer_size
assert apply_gating_layer(x=x, hidden_layer_size=hidden_layer_size)[0].shape == [batch_size, n_timesteps, hidden_layer_size]

In [9]:
#| output: asis
show_doc(apply_gating_layer)


---

### apply_gating_layer



Gated Linear Unit (GLU) layer

## Skip connection

Adds inputs to layer, ie "skip connection", and then implements layer normalisation [@ba2016layer].

In [10]:
def add_and_norm(
    x_list # List of input tensors (of the same dimension) for skip connection
    ):
    "Adds tensors with same dimensions and then normalises layer"
    tmp = layers.Add()(x_list)
    return layers.LayerNormalization()(tmp)


Example usage of skip connections + layer normalization:

In [11]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 100

# input dimensions: batches / timesteps / features
x1 = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 
x2 = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# output dimensions: batches / timesteps / features
x1x2 = add_and_norm(x_list=[x1, x2])
assert x1.shape == x1x2.shape

Mean values (normalised should be around 0):

In [12]:
x1.mean(axis=-1), x2.mean(axis=-1), x1x2.numpy().mean(axis=-1)

(array([[-0.21064889,  0.00084476,  0.11828817,  0.10758099,  0.13766904],
        [-0.10135074, -0.07045425,  0.14023677, -0.11775093, -0.02139493],
        [ 0.19945069,  0.15204023, -0.00915458, -0.05093143,  0.06814506]]),
 array([[-0.04798872, -0.03549833,  0.03819124,  0.06419377,  0.0571289 ],
        [ 0.03671484,  0.01313728,  0.24330115, -0.02981159, -0.13643608],
        [ 0.16745957,  0.15059631,  0.12545591,  0.04036627,  0.221975  ]]),
 array([[-2.3841857e-09,  3.5762786e-09, -3.5762786e-09,  1.9669534e-08,
          2.3841857e-09],
        [ 2.7418137e-08, -3.3974647e-08,  9.0897085e-09, -1.0430813e-08,
          8.3446503e-09],
        [-1.3113022e-08, -1.5497207e-08, -2.0861625e-08,  5.9604643e-10,
         -1.8775463e-08]], dtype=float32))

Standard deviation (normalised should be around 1):

In [13]:
x1.std(axis=-1), x2.std(axis=-1), x1x2.numpy().std(axis=-1)

(array([[0.95521739, 1.01235777, 1.06807431, 1.07700502, 1.00896732],
        [0.98399389, 0.96555043, 0.97562998, 0.91029426, 1.12211808],
        [0.90803724, 0.98626599, 0.95183459, 0.95716115, 1.00375778]]),
 array([[0.87385494, 0.97622118, 0.90593613, 0.91305024, 0.9836994 ],
        [0.98398415, 0.98993835, 0.99793171, 1.09443932, 1.08308257],
        [0.9164024 , 1.12422456, 1.18169786, 0.98498   , 1.0061887 ]]),
 array([[0.9996749 , 0.9997219 , 0.99976456, 0.9997423 , 0.9997195 ],
        [0.99968237, 0.9997152 , 0.9996925 , 0.9997782 , 0.99981135],
        [0.99963486, 0.9997779 , 0.99981385, 0.99975884, 0.9997358 ]],
       dtype=float32))

In [14]:
#| output: asis

show_doc(add_and_norm)

---

### add_and_norm

>      add_and_norm (x_list)

Adds tensors with same dimensions and then normalises layer

|    | **Details** |
| -- | ----------- |
| x_list | List of input tensors (of the same dimension) for skip connection |

## Gated residual network (GRN)

* The GRN is a key building block of the TFT
    * Helps keep information only from relevant input variables
    * Also keeps the model as simple as possible by only applying non-linearities when relevant
* $\text{GRN}_{\omega}(a, c)$:
    * *1st step*: $\eta_{2} = \text{ELU}(W_{2, \omega} a + b_{2, \omega} + W_{3, \omega} c)$, (where the additional context $c$ might be zero),
    * *2nd step*: $\eta_{1} = W_{1, \omega} \eta_{2} + b_{1, w}$,
    * *3rd step*: $\text{LayerNorm}(a + \text{GLU}_{\omega}(\eta_{1}))$
* $\text{ELU}(\cdot)$ is the Exponential Linear Unit activation function (@clevert2015fast)
    * Unlike ReLUs, ELUs allow for negative values, which pushes unit activations closer to zero at a lower computation complexity, and producing more accurate results
* $\text{LayerNorm}(\cdot)$ is the layer normalisation (@ba2016layer)

In [15]:
def gated_residual_network(
    x, # Network inputs
    hidden_layer_size:int, # Dimension of the GRN
    output_size:int|None=None, # Size of output layer (if None, same as `hidden_layer_size`)
    dropout_rate:float|None=None, # Dropout rate
    use_time_distributed:bool=True, # Apply the GRN across all timesteps?
    additional_context=None, # Additional context vector to use if relevant
    return_gate:bool=False #Whether to return GRN gate for diagnostic purposes
):
    "Applies the gated residual network (GRN) as defined in the paper"
    
    # Setup skip connection
    if output_size is None:
        output_size = hidden_layer_size
        skip = x
    else:
        linear = keras.layers.Dense(output_size)
        if use_time_distributed:
            linear = keras.layers.TimeDistributed(linear)
        skip = linear(x)

    # 1st step: eta2
    hidden = linear_layer(
        size=hidden_layer_size, # W2
        activation=None,
        use_time_distributed=use_time_distributed,
        use_bias=True # b2
    )(x)

    # "For instances without a context vector, the GRN simply treates the context input as zero - ie, $c = 0$ in Eq. 4"
    if additional_context is not None: # if c is != 0...
        hidden += linear_layer(
            size=hidden_layer_size, # W3
            activation=None,
            use_time_distributed=use_time_distributed,
            use_bias=False # no bias for additional context, since there already is bias from the "main" calculation of eta2
        )(additional_context)

    hidden = keras.layers.Activation('elu')(hidden)

    # 2nd step: eta1
    hidden = linear_layer(
        size=hidden_layer_size, # W1
        activation=None,
        use_time_distributed=use_time_distributed,
        use_bias=True # b1
    )(hidden)

    # 3rd step: concluding the GRN calculation
    gating_layer, gate = apply_gating_layer(
        x=hidden,
        hidden_layer_size=output_size,
        dropout_rate=dropout_rate,
        use_time_distributed=use_time_distributed,
        activation=None
    )

    GRN = add_and_norm([skip, gating_layer])

    if return_gate:
        return GRN, gate
    else:
        return GRN

Example usage of GRN:

In [16]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 100
hidden_layer_size = 16
output_size = 17

# input dimensions: batches / timesteps / features
x = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

grn = gated_residual_network(
    x=x,
    hidden_layer_size=hidden_layer_size,
    output_size=output_size,
    dropout_rate=0,
    use_time_distributed=True,
    additional_context=None,
    return_gate=False
)

# output dimensions: batches / timesteps / hidden_layer_size
assert grn.shape == [batch_size, n_timesteps, output_size]

In [17]:
#| output: asis

show_doc(gated_residual_network)

---

### gated_residual_network

>      gated_residual_network (x, hidden_layer_size:int,
>                              output_size:int|None=None,
>                              dropout_rate:float|None=None,
>                              use_time_distributed:bool=True,
>                              additional_context=None, return_gate:bool=False)

Applies the gated residual network (GRN) as defined in the paper

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| x |  |  | Network inputs |
| hidden_layer_size | int |  | Dimension of the GRN |
| output_size | int \| None | None | Size of output layer (if None, same as `hidden_layer_size`) |
| dropout_rate | float \| None | None | Dropout rate |
| use_time_distributed | bool | True | Apply the GLU across all timesteps? |
| additional_context | NoneType | None | Additional context vector to use if relevant |
| return_gate | bool | False | Whether to return GLU gate for diagnostic purposes |

# Attention components

* Attention mechanisms use relationships between keys $K \in \mathbf{R}^{N \times d_{attention}}$ and queries $Q \in \mathbf{R}^{N \times d_{attention}}$ to scale a vector of values $V \in \mathbf{R}^{N \times d_V}$: $\text{Attention}(Q, K, V) = A(Q, K) V$
    * $N$ is the number of timesteps going into the attention layer (the number of lags plus the number of periods to be forecasted)
    * $A(\cdot)$ is a normalisation function
        * After @vaswani2017attention, the canonical choice for $A(\cdot)$ is the scaled dot-product: $A(Q, K) = \text{Softmax}(\frac{Q K^{T}}{\sqrt{d_{attention}}} )$
    
* The TFT uses a modified attention head to enhance the explainability of the model
* Specifically, the transformer block (multi-head attention) is modified to:
    * share values in each head, and
    * employ additive aggregation of all heads
* More formally, compare the interpretable multi-head attention (used in this paper) with the canonical multi-head attention:
    * $\text{InterpretableMultiHead}(Q, K, V) = \tilde{H} W_{H}$, with:
        * $\begin{aligned}\tilde{H} &= \tilde{A}(Q, K) V W_V \\
        &= \{\frac{1}{m_H} \sum^{m_{H}}_{h=1} A(Q W^{(h)}_Q, K W^{(h)}_K) \} V W_V \\
        &= \frac{1}{m_H} \sum^{m_{H}}_{h=1} \text{Attention}(Q W^{(h)}_Q, K W^{(h)}_K, V W_V)
        \end{aligned}$
    * $\text{MultiHead}(Q, K, V) = [H_1, \dots, H_{m_H}] W_H$, with:
        * $H_h = \text{Attention}(Q W^{(h)}_Q, K W^{(h)}_K, V W_V^{(h)}) $

## Decoder mask for self-attention layer

In [18]:
def get_decoder_mask(
    self_attention_inputs # Inputs to the self-attention layer
):
    "Determines shape of decoder mask"
    len_s = keras.ops.shape(self_attention_inputs)[1] # length of inputs
    bs = keras.ops.shape(self_attention_inputs)[0] # batch shape
    mask = keras.ops.cumsum(keras.ops.eye(len_s), 1) #keras.backend.cumsum(np.eye(len_s, bs))

    ### warning: I had to manually implement some batch-wise shape here 
    ### because the new keras `eye` function does not have a batch_size arg.
    ### inspired by: https://github.com/tensorflow/tensorflow/blob/v2.14.0/tensorflow/python/ops/linalg_ops_impl.py#L30
    ### <hack>
    mask = keras.ops.expand_dims(mask, axis=0)    
    mask = keras.ops.tile(mask, (bs, 1, 1))
    ### </hack>

    return mask


Example usage of the decoder mask:

In [19]:
dec = get_decoder_mask(grn)

assert dec.shape == (batch_size, n_timesteps, n_timesteps)

Note that it produces an upper-triangular matrix of ones:

In [20]:
dec[0]

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)>

In [21]:
show_doc(get_decoder_mask)

---

### get_decoder_mask

>      get_decoder_mask (self_attention_inputs)

Determines shape of decoder mask

|    | **Details** |
| -- | ----------- |
| self_attention_inputs | Inputs to the self-attention layer |

## Scaled dot product attention layer

* This is the same as Eq. (1) of @vaswani2017attention 
    * except that in this case the dimension of the value vector is the same $d_{\text{model}}$ as for the query and key vectors
* As discussed in the paper, additive attention outperforms dot product attention for larger $d_{\text{model}}$ values, so the attention is scaled back to smaller values

In [22]:
class ScaledDotProductAttention():
    def __init__(
        self,
        training:bool=True, # Whether the layer is being trained or used in inference
        attention_dropout:float=0.0 # Will be ignored if `training=False`
    ):
        self.training = training
        self.dropout = keras.layers.Dropout(rate=attention_dropout)
        self.activation = keras.layers.Activation('softmax')

    def __call__(
        self,
        q, # Queries, tensor of shape (?, time, D_model)
        k, # Keys, tensor of shape (?, time, D_model)
        v, # Values, tensor of shape (?, time, D_model)
        mask # Masking if required (sets Softmax to very large value), tensor of shape (?, time, time)
    ):
        # returns Tuple (layer outputs, attention weights)
        scale = keras.ops.sqrt(keras.ops.cast(keras.ops.shape(k)[-1], dtype='float32'))
        attention = keras.ops.einsum("bij,bjk->bik", q, keras.ops.transpose(k, axes=(0, 2, 1))) / scale
        if mask is not None:
            mmask = keras.layers.Lambda(lambda x: (-1e9) * (1. - keras.ops.cast(x, 'float32')))(mask)
            attention = keras.layers.Add()([attention, mmask])
        attention = self.activation(attention)
        if self.training:
            attention = self.dropout(attention)
        output = keras.ops.einsum("btt,btd->bt", attention, v)
        return output, attention

Below is an example of how the `ScaledDotProductAttention` layer works:

In [23]:
#| code-fold: show

batch_size = 3
n_timesteps = 5
n_features = 13

# input dimensions: batches / timesteps / features
x_btf = np.random.randn(batch_size*n_timesteps*n_features).reshape([batch_size, n_timesteps, n_features]) 

# using the same vector for q, k and v just to simplify
q=keras.ops.cast(x_btf, 'float32')
k=keras.ops.cast(x_btf, 'float32')
v=keras.ops.cast(x_btf, 'float32')

Testing without masking:

In [24]:
#| code-fold: show

output, attention = ScaledDotProductAttention()(q=q, k=k, v=v, mask=None)
output, attention # both have shape (batch_size, n_timesteps)

(<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
 array([[-1.3595526 ,  2.3038948 , -0.9351378 , -2.1107376 , -0.10875011],
        [ 0.74828297,  1.2987878 ,  2.7947266 ,  3.3696222 ,  0.02006484],
        [-4.0625644 ,  7.9136066 ,  0.15763263,  0.387267  ,  0.5300054 ]],
       dtype=float32)>,
 <tf.Tensor: shape=(3, 5, 5), dtype=float32, numpy=
 array([[[8.67120624e-01, 7.71935284e-02, 1.09112160e-02, 3.08754463e-02,
          1.38991298e-02],
         [1.48252323e-01, 7.48296022e-01, 6.03066683e-02, 2.28216760e-02,
          2.03233790e-02],
         [4.45502140e-02, 1.28209814e-01, 7.00278938e-01, 7.39124417e-02,
          5.30485511e-02],
         [2.03909084e-01, 7.84784630e-02, 1.19554065e-01, 5.13916612e-01,
          8.41417834e-02],
         [5.25127314e-02, 3.99808772e-02, 4.90878299e-02, 4.81354706e-02,
          8.10283065e-01]],
 
        [[9.24033284e-01, 1.75150018e-02, 4.11768891e-02, 4.70066769e-03,
          1.25742713e-02],
         [9.32998657e-02, 6.94437802e-01

... and with masking:

In [25]:
output, attention = ScaledDotProductAttention()(q=q, k=k, v=v, mask=get_decoder_mask(q))
output, attention # both have shape (batch_size, n_timesteps)

(<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
 array([[-1.3595526 ,  2.704903  , -1.130431  , -3.5293167 , -0.1342125 ],
        [ 0.74828297,  1.4324336 ,  3.4899209 ,  3.8354926 ,  0.03641403],
        [-4.0625644 ,  7.9578185 ,  0.22934216,  0.414706  ,  0.56441283]],
       dtype=float32)>,
 <tf.Tensor: shape=(3, 5, 5), dtype=float32, numpy=
 array([[[8.67120624e-01, 7.71935284e-02, 1.09112160e-02, 3.08754463e-02,
          1.38991298e-02],
         [0.00000000e+00, 8.78541887e-01, 7.08034411e-02, 2.67939400e-02,
          2.38607973e-02],
         [0.00000000e+00, 0.00000000e+00, 8.46524537e-01, 8.93482491e-02,
          6.41271621e-02],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.59308362e-01,
          1.40691578e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          1.00000000e+00]],
 
        [[9.24033284e-01, 1.75150018e-02, 4.11768891e-02, 4.70066769e-03,
          1.25742713e-02],
         [0.00000000e+00, 7.65895724e-01

In [26]:
show_doc(ScaledDotProductAttention)

---

### ScaledDotProductAttention

>      ScaledDotProductAttention (training:bool=True,
>                                 attention_dropout:float=0.0)

Initialize self.  See help(type(self)) for accurate signature.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| training | bool | True | Whether the layer is being trained or used in inference |
| attention_dropout | float | 0.0 | Will be ignored if `training=False` |

### Softmax

A small detour to illustrate the softmax function. 

The $i^{\text{th}}$ element of $\text{Softmax}(x)$, with $x \in \mathbf{R}^K$ is:

$$
\text{Softmax}(x)_i = \frac{e^{x_i}}{\sum_{j=1}^K e^{x_j}}
$$

For example, see the values below for an input vector $x$ ($K=5$ in this example):

In [27]:
#| code-fold: show

x = np.array([-np.Inf, -1., 0., 1., 3.])
keras.layers.Activation('softmax')(x)
print("x = ", x)
print("exp(x) = ", np.exp(x))
print("denominator (sum of exp(x_j), j=1,...,K) = ", sum(np.exp(x)))
print("softmax(x) = ", np.exp(x) / sum(np.exp(x)))
print("sum of softmax(x)_j, j=1,...,K = ", sum(np.exp(x) / sum(np.exp(x))))

x =  [-inf  -1.   0.   1.   3.]
exp(x) =  [ 0.          0.36787944  1.          2.71828183 20.08553692]
denominator (sum of exp(x_j), j=1,...,K) =  24.171698192818155
softmax(x) =  [0.         0.01521943 0.0413707  0.11245721 0.83095266]
sum of softmax(x)_j, j=1,...,K =  1.0


As can be seen above, the softmax function really makes the largest numbers stand out from the rest.

Note also that $-\infty$ results in 0.

## Interpretable Multi-head attention

* When values are shared in each head and then are aggregated additively, each head still lcan learn different temporal patterns (from their own unique queries and keys), but with the same input values.
    * In other words, they can be interpreted as an ensemble over the attention weights
    * the paper doesn't mention this explicitly, but the ensemble is equally-weighted - maybe there is some performance to be gained by having some way to weight the different attention heads 🤔, such as having a linear layer combining them... will explore in the future

In [39]:
class InterpretableMultiHeadAttention():
    def __init__(
        self,
        n_head:int,
        d_model:int,
        training:bool=True, # Whether the layer is being trained or used in inference
        dropout:float=0.0 # Will be ignored if `training=False`
    ):
        self.n_head = n_head
        self.d_k = self.d_v = d_k = d_v = d_model # // n_head - the original model divides by number of heads
        self.training = training
        self.dropout = dropout

        # using the same value layer facilitates interpretability
        vs_layer = keras.layers.Dense(d_v, use_bias=False, name="Shared value")

        # creates list of queries, keys and values across heads
        self.qs_layers = self._build_layers(d_k, n_head)
        self.ks_layers = self._build_layers(d_k, n_head)
        self.vs_layers = [vs_layer for _ in range(n_head)]

        self.attention = ScaledDotProductAttention()
        self.w_o = keras.layers.Dense(d_v, use_bias=False, name="W_v") # W_v in Eqs. (14)-(16), output weight matrix to project internal state to the original TFT

    def __call__(
        self,
        q, # Queries, tensor of shape (?, time, D_model)
        k, # Keys, tensor of shape (?, time, D_model)
        v, # Values, tensor of shape (?, time, D_model)
        mask=None # Masking if required (sets Softmax to very large value), tensor of shape (?, time, time)
    ):
        heads = []
        attns = []
        for i in range(self.n_head):
            qs = self.qs_layers[i](q)
            ks = self.ks_layers[i](q)
            vs = self.vs_layers[i](v)
           
            head, attn = self.attention(qs, ks, vs, mask)
            if self.training:
                head = keras.layers.Dropout(self.dropout)(head)
            heads.append(head)
            attns.append(attn)

        outputs = keras.ops.mean(heads, axis=0) if self.n_head > 1 else head # H_tilde
        outputs = self.w_o(outputs)
        if self.training:
            outputs = keras.layers.Dropout(self.dropout)(outputs)

        return outputs, attn

    def _build_layers(self, d:int, n_head:int):
            return [keras.layers.Dense(d) for _ in range(n_head)]

In [40]:
imha = InterpretableMultiHeadAttention(n_head=8, d_model=16)

In [41]:
grn.shape # B, T, F

TensorShape([3, 5, 17])

In [42]:
mask = get_decoder_mask(grn)
mask.shape # B, T, T

TensorShape([3, 5, 5])

In [43]:
imha(grn, grn, grn, mask)

(<tf.Tensor: shape=(3, 16), dtype=float32, numpy=
 array([[ 1.6538911 , -1.5456989 , -0.15283048,  2.7386441 , -0.5256885 ,
          0.71322477,  2.6480474 , -1.675452  , -1.5537498 , -1.8390272 ,
          0.8307655 , -0.17164582,  1.3680466 ,  0.5958487 ,  2.9639983 ,
          2.0415192 ],
        [-0.01169664,  1.2011886 ,  0.8329091 ,  2.2552555 , -1.3456283 ,
         -0.4282118 ,  0.5777761 ,  0.19704199, -0.7254917 , -0.58820635,
         -1.2332759 , -0.4459725 , -0.6127497 ,  0.7004302 , -0.20625532,
          0.64137644],
        [-0.09603077,  0.3872428 , -0.8894279 ,  0.6738455 ,  0.2292835 ,
         -0.16534019,  0.4301936 , -0.746166  , -0.5417243 ,  0.15263456,
         -1.5832616 ,  0.12967631, -0.56263334,  0.46816152, -0.5202312 ,
         -0.59023654]], dtype=float32)>,
 <tf.Tensor: shape=(3, 5, 5), dtype=float32, numpy=
 array([[[0.11542334, 0.22207421, 0.3132397 , 0.25249708, 0.09676565],
         [0.        , 0.06326903, 0.50399   , 0.3407762 , 0.09196478],
   

# Time to build the TFT

In [59]:
class TemporalFusionTransformer():
    def __init__(
        self,
        # Data params
        time_steps:int,
        input_size:int,
        output_size:int,
        category_counts:int,
        n_workers:int, # Number of multiprocessing workers

        # TFT params
        input_obs_loc,
        static_input_loc,
        known_regular_input_idx,
        known_categorical_input_idx,
        column_definition,

        # Network params
        quantile:list=[0.1, 0.5, 0.9], # List of quantiles the model should forecast
        hidden_layer_size:int=30, # Size of hidden layer
        dropout_ratio:float=0.0, # Dropout ratio (between 0.0, inclusive, and less than 1.0)
        num_encoder_steps:int=4,
        num_stacks:int=4,
        num_heads:int=4,
        
        # Training params
        max_gradient_norm:float=1.0, # 
        learning_rate:float=0.001,
        minibatch_size:int=64,
        num_epochs:int=100,
        early_stopping_patience:int=5,
        use_gpu:bool=True
    ):
        self.time_steps = time_steps,
        self.input_size = input_size,
        self.output_size = output_size, # Number of periods to be forecasted
        self.category_counts = category_counts,
        self.n_workers = n_workers, # Number of multiprocessing workers
        
        self.input_obs_loc = input_obs_loc,
        self.static_input_loc = static_input_loc,
        self.known_regular_input_idx = known_regular_input_idx,
        self.known_categorical_input_idx = known_categorical_input_idx,
        self.column_definition = column_definition,

        self.quantile = quantile, # List of quantiles the model should forecast
        self.hidden_layer_size = hidden_layer_size, # Size of hidden layer
        self.dropout_ratio = dropout_ratio, # Dropout ratio (between 0.0, inclusive, and less than 1.0)
        self.num_encoder_steps = num_encoder_steps,
        self.num_stacks = num_stacks,
        self.num_heads = num_heads,
        
        self.max_gradient_norm = max_gradient_norm,
        self.learning_rate = learning_rate,
        self.minibatch_size = minibatch_size,
        self.num_epochs = num_epochs,
        self.early_stopping_patience = early_stopping_patience,
        self.use_gpu = use_gpu

        self.model = self.build_model()

    def __static_combine_and_mask(
        self, 
        embedding # Transformed static inputs
    ):
        # Return the tensor output for the variable selection network
        # In the paper, that would be the bottom right square in Fig.2

        # Add temporal features
        _, num_static, _ = embedding.get_shape().as_list() # (embeddings are $\xi_t^(1, \dots, \m_{\chi})$)
        flattened = keras.layers.Flatten()(embedding) # $\Xi_t$

        # Nonlinear transformation with the GRN
        mlp_outputs = gated_residual_network(
            x=flattened, # Network inputs
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
            output_size=num_static, # Size of output layer (if None, same as `hidden_layer_size`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all timesteps?
            additional_context=None, # Additional context vector to use if relevant
        ) 
        sparse_weights = keras.layers.Activation('softmax')(mlp_outputs)
        sparse_weights = keras.ops.expand_dims(sparse_weights, axis=-1) # $\upsilon_{\chi t}$
        # it's the sparse weights above that determine by how much the variable will be influencing the model

        transformed_embeddings = []
        for i in range(num_static):
            transformed_embeddings.append(gated_residual_network(
                x=embedding[:, i:i+1, :], # Network inputs
                hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
                output_size=self.hidden_layer_size, # Size of output layer (if None, same as `hidden_layer_size`)
                dropout_rate=self.dropout_rate, # Dropout rate
                use_time_distributed=False, # Apply the GRN across all timesteps?
            ))
        transformed_embedding = keras.ops.concatenate(transformed_embeddings, axis=1) # $\tilde{\xi_t^(1, \dots, \m_{\chi})}$

        combined = keras.layers.Multiply()(
            [sparse_weights, transformed_embedding]
        )
        
        static_vec = keras.ops.sum(combined, axis=1)

        return static_vec, sparse_weights

    def _build_base_graph(self):
        # Build the graph, defining the layers of the TFT
        
        all_inputs = keras.layers.Input(
            shape=(self.time_steps, self.input_size)
        )
        unknown_inputs, known_combined_layer, past_inputs, static_inputs \
            = self.get_tft_embeddings(all_inputs)

        # first we isolate the known future inputs and observed past inputs
        if unknown_inputs is not None:
            historical_inputs = keras.ops.concatenate([
                unknown_inputs[:, :self.num_encoder_steps, :],
                known_combined_layer[:, :self.num_encoder_steps, :],
                past_inputs[:, :self.num_encoder_steps, :]
            ], axis=1)
        else:
            historical_inputs = keras.ops.concatenate([
                known_combined_layer[:, :self.num_encoder_steps, :],
                past_inputs[:, :self.num_encoder_steps, :]
            ])
        
        # and then we isolate the known future inputs
        future_inputs = known_combined_layer[:, :self.num_encoder_steps, :]

        # static vars
        static_encoder, static_weights = self.__static_combine_and_mask(static_inputs)

        # static enrichment
        static_context_variable_selection = gated_residual_network(
            x=static_encoder, # Network inputs
            hidden_layer_size=self.hidden_layer_size, # Dimension of the GRN
            output_size=self.hidden_layer_size, # Size of output layer (if None, same as `hidden_layer_size`)
            dropout_rate=self.dropout_rate, # Dropout rate
            use_time_distributed=False, # Apply the GRN across all timesteps?
        )
        static_context_enrichment

    def build_model(self):
        # Build model and define training losses

        transformer_layer, all_inputs, self._attention_components = self._build_base_graph()
        outputs = keras.layers.TimeDistributed(
            keras.layers.Dense(self.output_size * len(self.quantiles))
        )(transformer_layer[Ellipsis, self.num_encoder_steps:, :])
        model = keras.Model(inputs=all_inputs, outputs=outputs)

In [60]:
tft = TemporalFusionTransformer(
    time_steps=12,
    input_size=20,
    output_size=4,
    category_counts=5,
    n_workers=2, # Number of multiprocessing workers

    # TFT params
    input_obs_loc=24,
    static_input_loc=24,
    known_regular_input_idx=24,
    known_categorical_input_idx=24,
    column_definition=None,
)

TypeError: cannot unpack non-iterable NoneType object

# References {.unnumbered}